In [5]:
import os
import json
import math
import sys
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

import wandb

from accelerate import Accelerator
from transformers import get_scheduler

from config import Config
from audiomodel import AudioProcessing
from audiodataset import AudioDataset, TestDataset

def make_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def wandb_init(cfg):
    wandb.init(
            # set the wandb project where this run will be logged
            project=cfg.wandb_project_name,
            
            # track hyperparameters and run metadata
            config={
            "learning_rate": cfg.learning_rate,
            "epochs": cfg.num_train_epochs,
            "batch_size": cfg.batch_size,
            }
    )
    
def save_checkpoint(cfg, model, result, best_loss, epoch=0):
    save_checkpoint = False
    with open("{}/summary.jsonl".format(cfg.output_dir), "a") as f:
        f.write(json.dumps(result) + "\n\n")
        
    if result["train_loss"] < best_loss:
      best_loss = result["train_loss"]
      save_checkpoint = True
      
    # 모델 상태 저장
    if save_checkpoint and cfg.checkpointing_steps == "best":
        torch.save(model.state_dict(), os.path.join(cfg.output_dir, "best.pth"))

    torch.save(model.state_dict(), os.path.join(cfg.output_dir, "last.pth"))
    torch.save(model.state_dict(), os.path.join(cfg.output_dir, f"epoch_{epoch}.pth"))

    return best_loss

def main():

    cfg = Config()
    
    accelerator = Accelerator(gradient_accumulation_steps=cfg.gradient_accumulation_steps)
    make_dir(cfg.output_dir)
    make_dir(cfg.generated_dir)
    wandb_init(cfg)
    
    #compression_model, lm = build_model(cfg)
    model = AudioProcessing(cfg)
    
    audio_dataset = AudioDataset(cfg, train=True) 
    eval_dataset = AudioDataset(cfg, train=False)
    test_dataset = TestDataset(cfg)

    audio_dataloader = DataLoader(audio_dataset, batch_size=cfg.batch_size, shuffle=True, num_workers=12)
    eval_dataloader = DataLoader(eval_dataset, batch_size=cfg.eval_batch_size, shuffle=False, num_workers=4)
    test_dataloader = DataLoader(test_dataset, batch_size=1)

    optimizer_parameters = [param for param in model.lm.parameters() if param.requires_grad]
    
    optimizer = torch.optim.AdamW(
        optimizer_parameters, lr=cfg.learning_rate,
        betas=(cfg.adam_beta1, cfg.adam_beta2),
        weight_decay=cfg.adam_weight_decay,
        eps=cfg.adam_epsilon,
    )
    
    
    num_update_steps_per_epoch = math.ceil(len(audio_dataloader) / cfg.gradient_accumulation_steps)
    if cfg.max_train_steps is None:
      cfg.max_train_steps = cfg.num_train_epochs * num_update_steps_per_epoch
    
    lr_scheduler = get_scheduler(
          name=cfg.lr_scheduler_type,
          optimizer=optimizer,
          num_warmup_steps=cfg.num_warmup_steps * cfg.gradient_accumulation_steps,
          num_training_steps=cfg.max_train_steps * cfg.gradient_accumulation_steps,
      )


    audio_dataloader, eval_dataloader, model, optimizer, lr_scheduler = accelerator.prepare(
        audio_dataloader, eval_dataloader, model, optimizer, lr_scheduler
    )

    starting_epoch, completed_steps, best_loss = 0, 0, np.inf
    progress_bar = tqdm(range(cfg.max_train_steps), disable=not accelerator.is_local_main_process)
    
    for epoch in range(starting_epoch, cfg.num_train_epochs):
        print(f"-------------------EPOCH{epoch}-------------------------" )
        total_loss, total_val_loss = 0, 0
        model.train()
        for batch_idx, (wav, descriptions, lengths) in enumerate(audio_dataloader):
          with accelerator.accumulate(model):
              loss = model(wav, descriptions, lengths)
              ppl =  torch.exp(loss)
              total_loss += loss.detach().float()
              accelerator.backward(loss)     
              optimizer.step()
              lr_scheduler.step()
              optimizer.zero_grad()
              
          if accelerator.sync_gradients:
              progress_bar.update(1)
              completed_steps += 1
            
        model.eval()
        for batch_idx, (wav, descriptions, lengths) in enumerate(eval_dataloader):
              loss = model(wav, descriptions, lengths)
              total_val_loss += loss  
    
        if accelerator.is_main_process:         
            result = {}
            result["epoch"] = epoch + 1,
            result["step"] = completed_steps
            result["train_loss"] = round(total_loss.item()/len(audio_dataloader), 4)
            result["valid_loss"] = round(total_val_loss.item()/len(eval_dataloader), 4)
            
            wandb.log(result)
            result_string = "Epoch: {}, Loss Train: {}, Valid: {}\n".format(epoch + 1, result["train_loss"], result["valid_loss"])    
            accelerator.print(result_string) 
            best_loss = save_checkpoint(cfg, model, result, best_loss, epoch)
            
            unwrapped_model = accelerator.unwrap_model(model)
            for test_step, batch in enumerate(test_dataloader):
                gen_audio = unwrapped_model.inference(batch)
                audio_filename = f"epoch_{epoch}_{test_step}.wav"
                unwrapped_model.save_audio(gen_audio, audio_filename, cfg)
             
    #wandb.finish()

In [6]:
#main()
from accelerate import notebook_launcher
notebook_launcher(main, num_processes=1)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Launching training on one GPU.


step,▁█
train_loss,█▁
valid_loss,▁█
step,92
train_loss,2.151
valid_loss,2.4137


/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")

  0%|          | 0/46000 [00:00<?, ?it/s]

-------------------EPOCH0-------------------------


...

Epoch: 1, Loss Train: 2.457, Valid: 2.4067

-------------------EPOCH1-------------------------



  0%|          | 92/46000 [01:21<6:00:00,  2.13it/s]

Epoch: 2, Loss Train: 2.151, Valid: 2.4137

-------------------EPOCH2-------------------------



  0%|          | 97/46000 [01:59<38:04:44,  2.99s/it]

KeyboardInterrupt: 

### Other

In [2]:
from pathlib import Path
import time
import tqdm
import json
import typing as tp
import pandas as pd
import glob2
import math
import omegaconf
import torch
from torch.nn import functional as F
import torch.nn as nn
import sys
import torchaudio
from torch.utils.data import Dataset, DataLoader

class Config:
    def __init__(self):
        self.sample_rate = 16000
        self.is_training = True
        self.duration = 1
        self.total_updates = 10000
        self.eval_steps = 4
        self.device = 'cuda'  # 'cuda' 또는 'cpu'
        self.batch_size = 24
        self.eval_batch_size = 4
        self.train_data_path = "/workspace/train_dataset.csv"
        self.eval_data_path = "/workspace/eval_dataset.csv"
        self.output_dir = "./output_dir"
        self.checkpointing_steps = "best"
        self.save_every = 10
        self.with_tracking = False
        self.text_encoder_name = None  # 나중에 설정
        self.snr_gamma = 5.0
        self.freeze_text_encoder = True
        self.uncondition = False
        self.learning_rate = 3e-5
        self.adam_beta1 = 0.9
        self.adam_beta2 = 0.999
        self.adam_weight_decay = 1e-2
        self.adam_epsilon = 1e-08
        self.gradient_accumulation_steps = 1
        self.num_train_epochs = 1000
        self.num_warmup_steps = 0
        self.max_train_steps = None
        self.lr_scheduler_type = "linear"
        self.resume_from_checkpoint = None #"/workspace/output_dir_batch48/last/" #None
        self.wandb_project_name = "audiogen-finetune-init-test1"
        self.wandb_id = None #"earnest-pond-52"
        self.resume_epoch = 0 #127
        self.dtype = "float32"
        
        self.update_audiocraft_config()
        

    def update(self, **kwargs):
        for key, value in kwargs.items():
            # 기존 속성에 값 할당하거나 새 속성 생성
            if not hasattr(self, key):
                print(key)
            setattr(self, key, value)
            
    def update_audiocraft_config(self):
        self.solver = None
        self.fsdp = None
        self.profiler = None
        self.deadlock = None
        self.dataset = None
        self.checkpoint = None
        self.generate = None
        self.evaluate = None
        self.optim = None
        self.schedule = None
        self.default = None
        self.defaults = None
        self.autocast = None
        self.autocast_dtype = None

        self.compression_model_checkpoint = None
        self.channels = None
        self.logging = None
        self.lm_model = None
        self.codebooks_pattern = None
        self.transformer_lm = None
        self.classifier_free_guidance = None
        self.attribute_dropout = None
        self.fuser = None
        self.conditioners = None
        self.datasource = None


In [3]:
class AudioDataset(Dataset):
    def __init__(self, audio_paths, device, target_sample_rate=44100, duration=3):
        import pandas as pd
        """
        Args:
            audio_files_list (list): List of paths to audio files.
            target_sample_rate (int): The sample rate to which audio should be resampled.
            frame_length (int): The frame length for slicing or padding audio.
        """
        self.audio_paths = audio_paths
        self.target_sample_rate = target_sample_rate
        self.duration = duration
        self.device = device

        self.df = pd.read_csv(self.audio_paths)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        
        data = self.df.iloc[idx] #self.audio_files_list[idx]
        audio_path = data['sliced_audio_path']
        description = data['description']

        # Load audio signal file
        from audiotools import AudioSignal
        wav = AudioSignal(audio_path)
        length = wav.signal_length

        # Encode audio signal as one long file
        wav.to_mono()
        wav.resample(self.target_sample_rate)

        if wav.duration < self.duration:
          pad_len = int(self.duration * self.target_sample_rate) - wav.signal_length
          wav.zero_pad(0, pad_len)
        elif wav.duration > self.duration:
          wav.truncate_samples(self.duration * self.target_sample_rate)


        return wav.audio_data.squeeze(1), description, length

class TestDataset(Dataset):
    def __init__(self, prompts):

        self.prompts = prompts


    def __len__(self):
        return len(self.prompts)

    def __getitem__(self, idx):

        return self.prompts[idx]

In [ ]:
from audiocraft.solvers import base, builders
from audiocraft.solvers.compression import CompressionSolver
from audiocraft import metrics as eval_metrics
from audiocraft import models
from audiocraft.data.audio_utils import normalize_audio
from audiocraft.modules.conditioners import JointEmbedCondition, SegmentWithAttributes, WavCondition, ConditioningAttributes
from audiocraft.utils.utils import get_dataset_from_loader, is_jsonable, warn_once
from audiocraft.models.loaders import load_compression_model, load_lm_model

In [7]:
class AudioProcessing(nn.Module):
    
    def __init__(self, cfg):
        super().__init__()  # 부모 클래스 초기화 호출
        self.cfg = cfg
        self.compression_model, self.lm  = self.build_model(self.cfg)
        self.to_float32()
        self.freeze_layers()

    def forward(self, wav, descriptions, lengths):
        from audiocraft.modules.conditioners import JointEmbedCondition, SegmentWithAttributes, WavCondition, ConditioningAttributes
        audio_tokens = self.process_audio_tokenizer(wav.to(self.cfg.device))
        audio_tokens, padding_mask = self.post_process_audio_tokenizer(audio_tokens, audio_lengths=lengths)
        
        attributes = [
            ConditioningAttributes(text={'description': description})
            for description in descriptions]
    
        model_output = self.lm.compute_predictions(audio_tokens, conditions=attributes, condition_tensors=None)  # type: ignore
        logits = model_output.logits
    
        mask = padding_mask & model_output.mask
        ce, ce_per_codebook = self.compute_cross_entropy(logits, audio_tokens, mask)
        
        return ce

    def build_model(self, cfg):
        from audiocraft.models.loaders import load_compression_model, load_lm_model
        """Instantiate models and optimizer."""
        
        compression_model = load_compression_model('facebook/audiogen-medium', device=cfg.device)
        lm = load_lm_model('facebook/audiogen-medium', device=cfg.device)
    
        return compression_model, lm

    def process_audio_tokenizer(self, wav):
        with torch.no_grad():
            audio_tokens, scale = self.compression_model.encode(wav)
        return audio_tokens

    def post_process_audio_tokenizer(self, audio_tokens, audio_lengths=None):
        padding_mask = torch.ones_like(audio_tokens, dtype=torch.bool, device=audio_tokens.device)

        audio_tokens = audio_tokens.clone()
        padding_mask = padding_mask.clone()
        token_sample_rate = self.compression_model.frame_rate
        B, K, T_s = audio_tokens.shape
        for i in range(B):
            valid_tokens = math.floor(audio_lengths[i] / self.cfg.sample_rate * token_sample_rate)
            audio_tokens[i, :, valid_tokens:] = self.lm.special_token_id
            padding_mask[i, :, valid_tokens:] = 0

        return audio_tokens, padding_mask

    def compute_cross_entropy(self, logits: torch.Tensor, targets: torch.Tensor, mask: torch.Tensor) -> tp.Tuple[torch.Tensor, tp.List[torch.Tensor]]:

        B, K, T = targets.shape
        assert logits.shape[:-1] == targets.shape
        assert mask.shape == targets.shape
        ce = torch.zeros([], device=targets.device)
        ce_per_codebook: tp.List[torch.Tensor] = []
        for k in range(K):
            logits_k = logits[:, k, ...].contiguous().view(-1, logits.size(-1))  # [B x T, card]
            targets_k = targets[:, k, ...].contiguous().view(-1)  # [B x T]
            mask_k = mask[:, k, ...].contiguous().view(-1)  # [B x T]
            ce_targets = targets_k[mask_k]
            ce_logits = logits_k[mask_k]
            q_ce = F.cross_entropy(ce_logits, ce_targets)
            ce += q_ce
            ce_per_codebook.append(q_ce.detach())
        # average cross entropy across codebooks
        ce = ce / K
        return ce, ce_per_codebook

    def audio_generate(self, condition_tensors, gen_duration=5):
        with torch.no_grad():
            total_gen_len = math.ceil(gen_duration * self.compression_model.frame_rate)
            gen_tokens = self.lm.generate(
                None, condition_tensors, max_gen_len=total_gen_len,
                num_samples=1)
            gen_audio = self.compression_model.decode(gen_tokens, None)

        return gen_tokens, gen_audio

    def inference(self, descriptions):
        #with torch.no_grad():
        from audiocraft.modules.conditioners import JointEmbedCondition, SegmentWithAttributes, WavCondition, ConditioningAttributes
        attributes = [
        ConditioningAttributes(text={'description': description})
        for description in descriptions]
        _, gen_audio = self.audio_generate(attributes, gen_duration=self.cfg.duration)
        
        return gen_audio

    def to_float32(self):
        # 모든 가중치를 FP32로 변환
        for param in self.lm.parameters():
            param.data = param.data.to(dtype=torch.float32)

    def freeze_layers(self, train_layers=12):
        for param in self.lm.parameters():
            param.requires_grad = False
            
        if train_layers > 0 :
            num_layers = len(self.lm.transformer.layers)
            
            for i in range(num_layers - train_layers, num_layers):
                for param in self.lm.transformer.layers[i].parameters():
                    param.requires_grad = True
                    
            for name, param in self.lm.named_parameters():
                if 'out_norm' in name or 'linears' in name:
                    param.requires_grad = True
                
            
        
        
            

In [4]:
# Extract discrete codes from EnCodec
def process_audio_tokenizer(wav):
  with torch.no_grad():
      audio_tokens, scale = compression_model.encode(wav)
  return audio_tokens

def post_process_audio_tokenizer(audio_tokens, audio_lengths=None, cfg=None):
  padding_mask = torch.ones_like(audio_tokens, dtype=torch.bool, device=audio_tokens.device)
  # replace encodec tokens from padded audio with special_token_id

  audio_tokens = audio_tokens.clone()
  padding_mask = padding_mask.clone()
  token_sample_rate = compression_model.frame_rate
  B, K, T_s = audio_tokens.shape
  for i in range(B):
      # take the last token generated from actual audio frames (non-padded audio)
      #math.floor(float(n_frames[i]) / sr[i] * token_sample_rate)
      valid_tokens = math.floor(audio_lengths[i] / cfg.sample_rate * token_sample_rate)
      audio_tokens[i, :, valid_tokens:] = lm.special_token_id
      padding_mask[i, :, valid_tokens:] = 0

  return audio_tokens, padding_mask

def _compute_cross_entropy(
      logits: torch.Tensor, targets: torch.Tensor, mask: torch.Tensor
    ) -> tp.Tuple[torch.Tensor, tp.List[torch.Tensor]]:
        """Compute cross entropy between multi-codebook targets and model's logits.
        The cross entropy is computed per codebook to provide codebook-level cross entropy.
        Valid timesteps for each of the codebook are pulled from the mask, where invalid
        timesteps are set to 0.

        Args:
            logits (torch.Tensor): Model's logits of shape [B, K, T, card].
            targets (torch.Tensor): Target codes, of shape [B, K, T].
            mask (torch.Tensor): Mask for valid target codes, of shape [B, K, T].
        Returns:
            ce (torch.Tensor): Cross entropy averaged over the codebooks
            ce_per_codebook (list of torch.Tensor): Cross entropy per codebook (detached).
        """
        B, K, T = targets.shape
        assert logits.shape[:-1] == targets.shape
        assert mask.shape == targets.shape
        ce = torch.zeros([], device=targets.device)
        ce_per_codebook: tp.List[torch.Tensor] = []
        for k in range(K):
            logits_k = logits[:, k, ...].contiguous().view(-1, logits.size(-1))  # [B x T, card]
            targets_k = targets[:, k, ...].contiguous().view(-1)  # [B x T]
            mask_k = mask[:, k, ...].contiguous().view(-1)  # [B x T]
            ce_targets = targets_k[mask_k]
            ce_logits = logits_k[mask_k]
            q_ce = F.cross_entropy(ce_logits, ce_targets)
            ce += q_ce
            ce_per_codebook.append(q_ce.detach())
        # average cross entropy across codebooks
        ce = ce / K
        return ce, ce_per_codebook

def audio_generate(condition_tensors, gen_duration=5):
    with torch.no_grad():
      total_gen_len = math.ceil(gen_duration * compression_model.frame_rate)
      gen_tokens = lm.generate(
          None, condition_tensors, max_gen_len=total_gen_len,
          num_samples=1)
      gen_audio = compression_model.decode(gen_tokens, None)

    return gen_tokens, gen_audio

In [ ]:
wav, text, length = next(iter(audio_dataloader))
audio_tokens = process_audio_tokenizer(wav.to(cfg.device))
print("Wav shape: ", wav.shape)
print("Token shape: ", audio_tokens.shape)
post_process_audio_tokenizer(audio_tokens, audio_lengths=length)

import torch

# 가정: model이라는 이름의 PyTorch 모델이 이미 정의되어 있음
for name, param in lm.named_parameters():
    print(f"Layer {name} has data type {param.dtype}")
    #break  # 모든 레이어를 표시하지 않고 첫 레이어에서 루프 중단

def check_requires_grad(model: torch.nn.Module):
    for name, module in model.named_children():
        for param_name, param in module.named_parameters():
            print(f"{name}.{param_name}: requires_grad = {param.requires_grad}")

# DAC 모델의 인스턴스를 생성한 후에 아래와 같이 사용할 수 있습니다:
# dac_instance = DAC(...)
check_requires_grad(lm)